In [ ]:
import os
os.getcwd()

# Предобработка данных
В файле prepare_dataset.py производиться подготовка меток, выделение валидационной выборки и аугментация тренировочных изображений. По умолчанию при запуске файла аугментация не производится, поскольку на это уходит несколько часов. Вы можете включить её, добавив параметр --augment_data 1.  
Все команды я запускал из терминала Jupyter Notebook, но можно и в текущем блокноте

In [1]:
!python ./prepare_dataset.py --data_dir ./participants 

Количество тренировочных изображений: 2002
Количество тестовых изображений: 855



Преобразование меток: 100%|##########| 2002/2002 [00:10<00:00, 199.95it/s]

Выделение набора валидации: 100%|##########| 400/400 [00:00<00:00, 1104.82it/s]


Выбрав модель, я обучил её на всём доступном датасете. Также для уменьшения числа ложноположительных ответов я дополнил его изображениями без меток, которые доступны в директории ./background_images. Они взяты из общедоступного набора данных Hotels-50K, с которым я раньше работал.  
В итоговом тренировочном наборе было 8216 изображений (2002 исходных + 5514 аугментированных + 700 без меток)

# Обучение
### Я использовал предобученную модель архитектуры YOLOv7. Ссылку на скачивание весов предобученной модели можно найти в [репозитории YOLOv7](https://github.com/WongKinYiu/yolov7), или можете просто скачать по [ссылке](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6e_training.pt) (318 Мб). Для запуска обучения скачайте её и поместите в каталог yolov7  

### Веса лучшей обученной мной модели можно скачать с моего Google диска по [ссылке](https://drive.google.com/u/0/uc?id=15tGlSHYF97BJHqWNy2SKQEqIG85AMoD_&export=download) (1.2 Гб). Для получения предсказания скачайте модель и также поместите её в каталог yolov7  

### Результаты обучения (показатели метрик по каждой эпохе) находятся в файле result.txt. За 64 часа на RTX 3090 было пройдено 153 эпохи обучения.  

## Замените содержимое dataset.yaml в папке data_for_yolo следующим содержимым. В коде данный файл создаётся автоматически с помощью библиотеки PyYaml, но такой формат не устраивает YOLOv7


In [3]:
yaml_content = """
train: ./data_for_yolo/data/images/train/
val: ./data_for_yolo/data/images/test/

nc: 5

names: ['car', 'head', 'face', 'human', 'carplate']
"""

In [5]:
!python ./train_aux.py --workers 8 --device 0 --batch-size 4 --data ./data_for_yolo/dataset.yaml --img 1280 1280 --epochs 300 --cfg ./cfg/training/yolov7-e6e.yaml --weights 'yolov7-e6e_training.pt' --name yolov7-e6e-custom

^C


# Обнаружение
Если файл detect запускается в Google Colab, можеть возникнуть ошибка в файле yolov7/utils/google_utils.py. Для её обхода достаточно закоментировать строку 31 файла yolov7/utils/google_utils.py:  
```tag = subprocess.check_output('git tag', shell=True).decode().split()[-1]```

In [1]:
!python ./detect.py --weights ./yolov7.pt --device 0 --source ./participants/test/images --img-size 1280 --conf-thres 0.05 --save-txt --save-conf

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='0', exist_ok=False, img_size=1280, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=True, save_txt=True, source='./participants/test/images', update=False, view_img=False, weights=['./yolov7.pt'])
Fusing layers... 
IAuxDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 carplate, Done. (30.1ms) Inference, (10.0ms) NMS
 The image with the result is saved in: runs\detect\exp2\0_23-11-2021_02-18-50_PM.jpg
1 carplate, Done. (40.3ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp2\0_23-11-2021_02-18-56_PM.jpg
1 car, 1 carplate, Done. (33.2ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp2\0_23-11-2021_02-19-19_PM.jpg
1 carplate, Done. (30.1ms) Inference, (8.0ms) NMS
 The image with the result is saved in: runs\detect\exp2\0_23-11-2021_02-20-47_PM.jpg
1 carp

YOLOR  v0.1-105-g064c71e torch 1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

C:\Users\mmill\AppData\Roaming\Python\Python38\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model Summary: 819 layers, 164893160 parameters, 0 gradients, 225.5 GFLOPS


In [3]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import random


def get_soliton_labels_df_simple_solution(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']            
        for line in data:
            val = [float(i) for i in line.split()]
            cls, xywh, conf = int(val[0]), val[1:5], val[5]
            center_x, center_y, width, height = xywh
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            simple_solution.append([img_name, cls, conf, xmin, xmax, ymin, ymax])
    return simple_solution

In [4]:
simple_solution = get_soliton_labels_df_simple_solution('runs/detect/exp/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [5]:
simple_solution.to_csv("sample_solution.csv", sep=';', index=False)